In [4]:
from google.cloud import storage
import os

In [5]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/mnt/c/Users/Andres/Documents/api-keys/tlac-vision/tlac-vision-c0786b53c370.json"

In [24]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))

In [33]:
bucket_name = 'tlac-book-covers'
folder_name = 'unprocessed'

source_file_name = 'images/' + os.listdir('./images')[0]

destination_blob_name = folder_name + '/' + os.listdir('./images')[0]

In [39]:
def copy_blob(bucket_name, blob_name, new_bucket_name, new_blob_name):
    """Copies a blob from one bucket to another with a new name."""
    storage_client = storage.Client()
    source_bucket = storage_client.get_bucket(bucket_name)
    source_blob = source_bucket.blob(blob_name)
    destination_bucket = storage_client.get_bucket(new_bucket_name)

    new_blob = source_bucket.copy_blob(
        source_blob, destination_bucket, new_blob_name)
    
    source_blob.delete()

    print('Blob {} in bucket {} copied to blob {} in bucket {}.'.format(
        source_blob.name, source_bucket.name, new_blob.name,
        destination_bucket.name))
    print('Blob {} deleted.'.format(source_blob))

In [38]:
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()

    print('Blob {} deleted.'.format(blob_name))

In [40]:
source_file_name = folder_name + '/' + os.listdir('./images')[0]
destination_blob_name = 'successful' + '/' + os.listdir('./images')[0]

copy_blob(bucket_name, source_file_name, bucket_name, destination_blob_name)

Blob unprocessed/chopra.jpeg in bucket tlac-book-covers copied to blob successful/chopra.jpeg in bucket tlac-book-covers.
Blob <Blob: tlac-book-covers, unprocessed/chopra.jpeg> deleted.


In [34]:
upload_blob(bucket_name, source_file_name, destination_blob_name)

File images/chopra.jpeg uploaded to unprocessed/chopra.jpeg.


In [26]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

/mnt/c/Users/Andres/Documents/api-keys/tlac-vision/tlac-vision-c0786b53c370.json


In [29]:
!pwd

/mnt/c/Users/Andres/repos/book_cover_reader


In [2]:
import os

os.getcwd()

'/mnt/c/Users/Andres/repos/book_cover_reader'

In [35]:
type(os.listdir('./images'))

list

In [41]:
def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    blobs = bucket.list_blobs()

    for blob in blobs:
        print(blob.name)

In [42]:
list_blobs(bucket_name)

failed/
successful/
successful/chopra.jpeg
unprocessed/


In [43]:
def list_blobs_with_prefix(bucket_name, prefix, delimiter=None):
    """Lists all the blobs in the bucket that begin with the prefix.

    This can be used to list all blobs in a "folder", e.g. "public/".

    The delimiter argument can be used to restrict the results to only the
    "files" in the given "folder". Without the delimiter, the entire tree under
    the prefix is returned. For example, given these blobs:

        /a/1.txt
        /a/b/2.txt

    If you just specify prefix = '/a', you'll get back:

        /a/1.txt
        /a/b/2.txt

    However, if you specify prefix='/a' and delimiter='/', you'll get back:

        /a/1.txt

    """
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    blobs = bucket.list_blobs(prefix=prefix, delimiter=delimiter)

    print('Blobs:')
    for blob in blobs:
        print(blob.name)

    if delimiter:
        print('Prefixes:')
        for prefix in blobs.prefixes:
            print(prefix)

In [47]:
list_blobs_with_prefix(bucket_name, 'successful/')

Blobs:
successful/
successful/chopra.jpeg


In [6]:
import json

with open('vision_request.json') as f:
    vs_req = json.load(f)

In [7]:
vs_req

{'requests': [{'features': [{'type': 'TEXT_DETECTION'}],
   'image': {'source': {'imageUri': 'gs://tlac-book-covers/REPLACE'}}}]}

In [8]:

def update_image_location(req_dict, location):
    req_dict.get('requests')[0].get('image').get('source').update({'imageUri': location})
    return req_dict


In [9]:
vs_req = update_image_location(vs_req, 'gs://tlac-book-covers/chopra.jpeg')
vs_req

{'requests': [{'features': [{'type': 'TEXT_DETECTION'}],
   'image': {'source': {'imageUri': 'gs://tlac-book-covers/chopra.jpeg'}}}]}

In [10]:
algo = json.dumps(vs_req)
algo

'{"requests": [{"features": [{"type": "TEXT_DETECTION"}], "image": {"source": {"imageUri": "gs://tlac-book-covers/chopra.jpeg"}}}]}'

In [13]:
import requests

hed = {'Authorization': 'Bearer ' + token}

rs = requests.post("https://vision.googleapis.com/v1/images:annotate", data=algo, headers=hed)

In [14]:
rs.content

b'{\n  "responses": [\n    {\n      "textAnnotations": [\n        {\n          "locale": "es",\n          "description": "DEEPAK\\nCHOPRA\\n\xc2\xbfTIENE\\nFUTURO\\nDIOS?\\n",\n          "boundingPoly": {\n            "vertices": [\n              {\n                "x": 43,\n                "y": 388\n              },\n              {\n                "x": 717,\n                "y": 388\n              },\n              {\n                "x": 717,\n                "y": 1173\n              },\n              {\n                "x": 43,\n                "y": 1173\n              }\n            ]\n          }\n        },\n        {\n          "description": "DEEPAK",\n          "boundingPoly": {\n            "vertices": [\n              {\n                "x": 64,\n                "y": 444\n              },\n              {\n                "x": 700,\n                "y": 388\n              },\n              {\n                "x": 708,\n                "y": 485\n              },\n          

In [44]:
from google.cloud import vision

client = vision.ImageAnnotatorClient()
response = client.text_detection({'source': {'image_uri': 'gs://tlac-book-covers-todo/chopra.jpeg'},})

In [45]:
response

text_annotations {
  locale: "es"
  description: "DEEPAK\nCHOPRA\n\302\277TIENE\nFUTURO\nDIOS?\n"
  bounding_poly {
    vertices {
      x: 43
      y: 388
    }
    vertices {
      x: 717
      y: 388
    }
    vertices {
      x: 717
      y: 1173
    }
    vertices {
      x: 43
      y: 1173
    }
  }
}
text_annotations {
  description: "DEEPAK"
  bounding_poly {
    vertices {
      x: 64
      y: 444
    }
    vertices {
      x: 700
      y: 388
    }
    vertices {
      x: 708
      y: 485
    }
    vertices {
      x: 72
      y: 541
    }
  }
}
text_annotations {
  description: "CHOPRA"
  bounding_poly {
    vertices {
      x: 43
      y: 559
    }
    vertices {
      x: 711
      y: 515
    }
    vertices {
      x: 717
      y: 610
    }
    vertices {
      x: 49
      y: 654
    }
  }
}
text_annotations {
  description: "\302\277"
  bounding_poly {
    vertices {
      x: 150
      y: 725
    }
    vertices {
      x: 203
      y: 724
    }
    vertices {
      x: 206

In [46]:
from google.protobuf.json_format import MessageToJson
serialized = MessageToJson(response)
algo = json.loads(serialized)
algo

{'textAnnotations': [{'locale': 'es',
   'description': 'DEEPAK\nCHOPRA\n¿TIENE\nFUTURO\nDIOS?\n',
   'boundingPoly': {'vertices': [{'x': 43, 'y': 388},
     {'x': 717, 'y': 388},
     {'x': 717, 'y': 1173},
     {'x': 43, 'y': 1173}]}},
  {'description': 'DEEPAK',
   'boundingPoly': {'vertices': [{'x': 64, 'y': 444},
     {'x': 700, 'y': 388},
     {'x': 708, 'y': 485},
     {'x': 72, 'y': 541}]}},
  {'description': 'CHOPRA',
   'boundingPoly': {'vertices': [{'x': 43, 'y': 559},
     {'x': 711, 'y': 515},
     {'x': 717, 'y': 610},
     {'x': 49, 'y': 654}]}},
  {'description': '¿',
   'boundingPoly': {'vertices': [{'x': 150, 'y': 725},
     {'x': 203, 'y': 724},
     {'x': 206, 'y': 828},
     {'x': 153, 'y': 830}]}},
  {'description': 'TIENE',
   'boundingPoly': {'vertices': [{'x': 217, 'y': 720},
     {'x': 581, 'y': 710},
     {'x': 584, 'y': 821},
     {'x': 220, 'y': 831}]}},
  {'description': 'FUTURO',
   'boundingPoly': {'vertices': [{'x': 103, 'y': 870},
     {'x': 630, 'y': 

In [52]:
print(algo.get('textAnnotations')[0].get('description'))
algo2 = algo.get('textAnnotations')[0].get('description')
algo2

DEEPAK
CHOPRA
¿TIENE
FUTURO
DIOS?



'DEEPAK\nCHOPRA\n¿TIENE\nFUTURO\nDIOS?\n'

In [137]:
print("DEEPAK\nCHOPRA\n\bfTIENE\nFUTURO\nDIOS?\n")

DEEPAK
CHOPRA
Â¿TIENE
FUTURO
DIOS?



In [136]:
print("DEEPAK\nCHOPRA\n\302\277TIENE\nFUTURO\nDIOS?\n")

DEEPAK
CHOPRA
Â¿TIENE
FUTURO
DIOS?



In [18]:
query = "DEEPAK\nCHOPRA\n\302\277TIENE\nFUTURO\nDIOS?\n"
query = query.replace('\n', '+')

In [51]:
response = requests.get('https://www.googleapis.com/books/v1/volumes?q='+algo2)
response.status_code

my_str = response.content
new_str = my_str.decode('utf-8') # Decode using the utf-8 encoding
print(new_str)

{
 "kind": "books#volumes",
 "totalItems": 1632,
 "items": [
  {
   "kind": "books#volume",
   "id": "Fb-6CgAAQBAJ",
   "etag": "mqoSNqNtWyM",
   "selfLink": "https://www.googleapis.com/books/v1/volumes/Fb-6CgAAQBAJ",
   "volumeInfo": {
    "title": "¿Tiene futuro Dios?",
    "subtitle": "Un enfoque práctico para la espiritualidad de nuestro tiempo",
    "authors": [
     "Deepak Chopra"
    ],
    "publisher": "Grijalbo",
    "publishedDate": "2015-11-13",
    "description": "Por sí misma, la fe no desemboca en Dios, pero hace algo mucho más prolongado: hace posible a Dios. Por tres décadas, Deepak Chopra ha inspirado a millones de personas a través de su escritura y su enseñanza. Con ¿Tiene futuro Dios? nos invita a un viaje del espíritu, dándonos una vía práctica para entender a ese ser superior y dónde estamos. Ahora es el momento de tomar fuerzas. Es el momento de renovarnos. Ahora es el futuro. Dios está en problemas. El fortalecimiento del movimiento ateísta esparcido por Richar